In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import pickle
from collections import defaultdict

from torch.utils.data import Dataset, DataLoader
from Mmetrics import *

import LTR
import datautil
import permutationgraph
import DTR
import EEL
import PPG
import PL

def df2ds(df_path):
    with open(df_path, 'rb') as f:
        df = pickle.load(f)
    ds = df.to_dict(orient='list')
    for k in ds:
        ds[k] = np.array(ds[k])
    ds['dlr'] = np.concatenate([np.zeros(1), np.where(np.diff(ds['qid'])==1)[0]+1, np.array([ds['qid'].shape[0]])]).astype(int)
    return type('ltr', (object,), ds)


def dict2ds(df_path):
    with open(df_path, 'rb') as f:
        ds = pickle.load(f)
    return type('ltr', (object,), ds)

ds2019 = df2ds('nLTR2019.df')
ds2020 = df2ds('nLTR2020.df')
dsmslr = dict2ds('nLTRMSLR_dict.df')

def valid_queries(ds):
    dtr, eel = [], []
    groups = np.unique(ds.g)
    for qid in range(ds.dlr.shape[0] - 1):
        s, e = ds.dlr[qid:qid+2]
        lv = ds.lv[s:e]
        g = ds.g[s:e]
        z = False
        for group in groups:
            if lv[g==group].sum() == 0:
                z = True
                break
        if not z:
            dtr.append(qid)
        if len(np.unique(ds.g[s:e])) > 1:
            eel.append(qid)
            
    return {'DTR':np.array(dtr), 'EEL':np.array(eel)}

ds2019.valids = valid_queries(ds2019)
ds2020.valids = valid_queries(ds2020)
dsmslr.valids = {'EEL':np.arange(dsmslr.dlr.shape[0]-1), 'DTR':np.arange(dsmslr.dlr.shape[0]-1)}

def evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    
    if metric == 'EEL':
        objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 2)
    else:
        objective_ins = DTR.DTR(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure)
        
    
    osl = e - s
    argsort = lv[s:e].argsort()[::-1]
    idcg = ((2.**lv[s:e][argsort][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum()
    ndcg = 0
    for i in range(sessions_cnt):
        ndcg += ((2.**lv[s:e][permutation[i*osl:(i+1)*osl]-(i*osl)][:min(osl,10)] - 1.) / (np.log2(2+np.arange(min(osl,10))))).sum() / idcg
        
    return objective_ins.eval(permutation), ndcg / sessions_cnt
 
def evaluate_all(metric, valids, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res, ndcgs = [], [], [], []
#     for qid in range(dlr.shape[0] - 1):
    for qid in valids:
        s,e = dlr[qid:qid+2]
#         if len(np.unique(g[s:e])) == 1:
#             continue
        out1, ndcg = evaluate_one(metric, qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
#         eel, eer, eed = out1
        eel = out1
        eel_res.append(eel)
#         eer_res.append(eer)
#         eed_res.append(eed)
        ndcgs.append(ndcg)
    return np.array(eel_res), np.array(ndcgs)
#     return np.array(eel_res), np.array(eer_res), np.array(eed_res), np.array(ndcgs)

def estimated_evaluate_one(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt):
    s, e = dlr[qid:qid+2]
    permutation = output_permutation[qid]
    lv_s, g_s, sorted_docs_s, dlr_s = \
        EEL.copy_sessions(y=lv[s:e], g=g[s:e], sorted_docs=lv[s:e].argsort()[::-1], sessions=sessions_cnt)
    objective_ins = EEL.EEL(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure, grade_levels = 5)
    return objective_ins.eval_detailed(permutation)
 
def estimated_evaluate_all(lv, g, dlr, output_permutation, exposure, sessions_cnt):
    eel_res, eer_res, eed_res = [], [], []
    for qid in range(dlr.shape[0] - 1):
        s,e = ds.tedlr[qid:qid+2]
        if len(np.unique(g[s:e])) == 1:
            continue
        eel, eer, eed = estimated_evaluate_one(qid, lv, g, dlr, output_permutation, exposure, sessions_cnt)
        eel_res.append(eel)
        eer_res.append(eer)
        eed_res.append(eed)
    return np.array(eel_res), np.array(eer_res), np.array(eed_res)

In [2]:

import matplotlib as mpl
import matplotlib.pyplot as plt
# mpl.rcParams['pdf.fonttype'] = 42
# mpl.use('pgf')
# mpl.rc('font',family='Times New Roman')



legends = [True, True]

def plot_per_sessions(title, df):
    df = df.loc[:,['learner', 'sessions', 'Fairness', 'NDCG']]
    ptbl = df.pivot_table(index='sessions', columns='learner', values=['Fairness', 'NDCG'])

    # print(ptbl.Fairness)
    if len(ptbl.Fairness.columns) < 3:
        return
    
    global legends
    legend_ind = 0
    if title[2]:
        legend_ind = 1
    ax = ptbl.Fairness.rename(columns={'PL_nointra':'PL', 'PPG_nointra':'PPG', 'PPG_intra': 'PPG + intra'})\
        [['PL', 'PPG', 'PPG + intra']]\
        .plot(fontsize=22, xlabel='', legend=legends[legend_ind], title=f'Fairness_{title[0]}_{title[1]}_{title[2]}')
    if legends[legend_ind]:
        plt.legend(fontsize=24)
        legends[legend_ind] = False
    # ptbl.NDCG.plot()
#     fig = ax.get_figure()
#     fig.savefig(f'../sections/figure/Fairness_{title[0]}_{title[1]}_{title[2]}.pdf',bbox_inches='tight', pad_inches=0)
#     plt.close()
    
    ndcg_ylim = {2019:(0.745,0.82), 2020:(0.335, 0.397)}
    ax = ptbl.NDCG.rename(columns={'PL_nointra':'PL', 'PPG_nointra':'PPG', 'PPG_intra': 'PPG + intra'})\
        [['PL', 'PPG', 'PPG + intra']]\
        .plot(fontsize=22, xlabel='', ylim=ndcg_ylim[title[0]] if not title[2] else None, legend=False, title=f'NDCG_{title[0]}_{title[1]}_{title[2]}')
    # plt.legend(fontsize=24) 
    # ptbl.NDCG.plot()
#     fig = ax.get_figure()
#     fig.savefig(f'../sections/figure/NDCG_{title[0]}_{title[1]}_{title[2]}.pdf',bbox_inches='tight', pad_inches=0)
#     plt.close()


In [23]:
import pickle
with open('trec_32_PL.pkl', 'rb') as f:
    df_PL = pickle.load(f)

with open('trec_32_new.pkl', 'rb') as f:
    df_PPG = pickle.load(f)

df = df_PL.append(df_PPG, ignore_index=True)

In [29]:
def test(validid):
    qid = ds2020.valids['DTR'][validid]
    s, e = ds2020.dlr[qid:qid+2]

    # print(lv_1[qid])
    # for i in range(4):
    #     print('\t', lv_4[qid][i*(e-s):(i+1)*(e-s)] - i*(e-s))


    exposure = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.dlr).max()+2)])

    lv_s, g_s, sorted_docs_s, dlr_s = \
            EEL.copy_sessions(y=ds2020.lv[s:e], g=ds2020.g[s:e], sorted_docs=ds2020.lv[s:e].argsort()[::-1], sessions=4)
    y_s, g_s, sorted_docs_s, dlr_s = \
            EEL.copy_sessions(y=ds2020.y_pred[s:e], g=ds2020.g[s:e], sorted_docs=ds2020.lv[s:e].argsort()[::-1], sessions=4)


    objective_ins_4 = DTR.DTR(y_pred = lv_s, g = g_s, dlr = dlr_s, exposure=exposure)
    objective_ins_1 = DTR.DTR(y_pred = ds2020.lv[s:e], g = ds2020.g[s:e], dlr = np.array([0,e-s]), exposure=exposure)
    print('true labels:')
    print(qid, objective_ins_1.eval(lv_1[qid]), objective_ins_4.eval(lv_4[qid]))

    for i in range(4):
        print('\t', objective_ins_1.eval(lv_4[qid][i*(e-s):(i+1)*(e-s)] - i*(e-s)))
       
    print('ltr output:')
    print(qid, objective_ins_1.eval(y_1[qid]), objective_ins_4.eval(y_4[qid]))

    for i in range(4):
        print('\t', objective_ins_1.eval(y_4[qid][i*(e-s):(i+1)*(e-s)] - i*(e-s)))
    
    objective_ins_4_y = DTR.DTR(y_pred = y_s, g = g_s, dlr = dlr_s, exposure=exposure)
    objective_ins_1_y = DTR.DTR(y_pred = ds2020.y_pred[s:e], g = ds2020.g[s:e], dlr = np.array([0,e-s]), exposure=exposure)
    
    print('estimated fairness:')
    print(qid, objective_ins_1_y.eval(y_1[qid]), objective_ins_4_y.eval(y_4[qid]))

    for i in range(4):
        print('\t', objective_ins_4_y.eval(y_4[qid][i*(e-s):(i+1)*(e-s)] - i*(e-s)))

In [3]:
import pickle
import os

def read_results(directory = '/ivi/ilps/personal/avardas/_data/PPG'):
    files = os.listdir(directory)
    res = {}
    for file in files:
        if 'pkl' not in file:
            continue
        with open(f'{directory}/{file}', 'rb') as f:
            res[file[:-12]] = pickle.load(f)
    return res
    
res = read_results(directory = '/ivi/ilps/personal/avardas/_data/PPG/n/mslr')
res.keys()

dict_keys(['nointra_ltroutput_1_1_mslr_EEL', 'nointra_PL_4_1_mslr_EEL', 'nointra_RAND_4_1_mslr_DTR', 'nointra_PPG_1_1_mslr_DTR', 'nointra_ltroutput_1_1_mslr_DTR', 'nointra_PL_4_1_mslr_DTR', 'nointra_RAND_4_1_mslr_EEL', 'nointra_PPG_1_1_mslr_EEL', 'intra_PPG_4_1_mslr_EEL', 'nointra_ltroutput_2_1_mslr_EEL', 'nointra_PL_32_1_mslr_DTR', 'nointra_ltroutput_8_1_mslr_DTR', 'nointra_PL_16_1_mslr_EEL', 'nointra_PPG_2_1_mslr_DTR', 'nointra_ltroutput_2_1_mslr_DTR', 'intra_PPG_4_1_mslr_DTR', 'nointra_ltroutput_8_1_mslr_EEL', 'nointra_PL_32_1_mslr_EEL', 'nointra_PL_16_1_mslr_DTR', 'nointra_PPG_2_1_mslr_EEL', 'nointra_PL_2_1_mslr_EEL', 'nointra_RAND_2_1_mslr_DTR', 'nointra_ltroutput_32_1_mslr_EEL', 'nointra_PL_8_1_mslr_DTR', 'nointra_RAND_8_1_mslr_EEL', 'nointra_ltroutput_16_1_mslr_DTR', 'intra_PPG_1_1_mslr_EEL', 'nointra_PL_2_1_mslr_DTR', 'nointra_RAND_2_1_mslr_EEL', 'nointra_ltroutput_32_1_mslr_DTR', 'nointra_PL_8_1_mslr_EEL', 'nointra_RAND_8_1_mslr_DTR', 'nointra_ltroutput_16_1_mslr_EEL', 'intra_

In [37]:
print(res['intra_PPG_4_1_DTR'].keys()) 

dict_keys(['2020_0.1_16_0', '2019_0.1_16_0', '2020_0.1_16_1', '2019_0.1_16_1'])


In [4]:

from tqdm.notebook import tqdm

def get_df_from_results(res):
    exposure2020 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2020.dlr).max()+2)])
    exposure2019 = np.array([1./np.log2(2+i) for i in range(1,np.diff(ds2019.dlr).max()+2)])
    exposuremslr = np.array([1./np.log2(2+i) for i in range(1,np.diff(dsmslr.dlr).max()+2)])


    dfl = []
    for alg in tqdm(res, leave=False):
        _res = res[alg]
#         if 'PPG_single_1_2' in alg:
#             continue
#         print(alg)
        alg_params = alg.split('_')
    #     print(alg, '-'*30)
        sessions_cnt = int(alg_params[2])
        metric = alg_params[-1]
        learner = alg_params[1] + '_' + alg_params[0]
#         if len(alg_params) > 5:
#             learner = alg_params[1] + alg_params[2] + '_' + alg_params[0]

        for key in tqdm(_res, leave=False):
            if '2019' in key:
                year = 2019
                ds = ds2019
                exposure = exposure2019
            elif '2020' in key:
                year = 2020
                ds = ds2020
                exposure = exposure2020
            else:
                year = 'mslr'
                ds = dsmslr
                exposure = exposuremslr

            
            lr = key.split('_')[-3]
            samples = int(key.split('_')[-2])
            ind = int(key.split('_')[-1])
            eel_res, ndcg = evaluate_all(metric, ds.valids[metric], ds.lv, ds.g, ds.dlr, _res[key], exposure, sessions_cnt=sessions_cnt)
            dfl.append({'ind':ind, 'year':year, 'metric':metric, 'learner':learner, 'true_labels':'lv' in key,
            'samples':samples, 'sessions':sessions_cnt, 'lr':lr, 'Fairness':eel_res.mean(), 'NDCG':ndcg.mean()})

    return pd.DataFrame(dfl)


df2 = get_df_from_results(res)
len(df2)

189

In [5]:
import pickle
with open('mslr_n.pkl', 'wb') as f:
    pickle.dump(df2, f)

In [9]:
def print_df(df, sessions):
    gdf = df.loc[df.sessions==sessions].groupby(['year', 'true_labels'])

    for n, d in gdf:
        d = d.loc[:,['learner', 'metric', 'Fairness', 'NDCG']]
        ptbl = d.pivot_table(index='learner', columns=['metric'], values=['Fairness', 'NDCG'], aggfunc=np.mean)
        print(n)
        print(ptbl)
#     print(ptbl.to_csv(header=False, index=True, float_format='%.3f').replace(',','&').replace('PL_nointra', 'PL').replace('PPG_intra', 'PPG + intra').replace('\n', '\\\\\n&                      & '))
    

In [10]:
for s in [1,2,4,8,16,32]:
    print(s, '='*30)
    print_df(df2, s)

1 ==============================
('mslr', False)
                   Fairness                NDCG          
metric                  DTR       EEL       DTR       EEL
learner                                                  
RAND_nointra       0.930552  0.139894  0.390617  0.394529
ltroutput_nointra  0.823925  0.114186  0.572315  0.572315
('mslr', True)
                   Fairness               NDCG         
metric                  DTR       EEL      DTR      EEL
learner                                                
RAND_nointra       0.931230  0.141152  0.39276  0.39285
ltroutput_nointra  0.548063  0.027232  1.00000  1.00000
2 ==============================
('mslr', False)
                   Fairness                NDCG          
metric                  DTR       EEL       DTR       EEL
learner                                                  
RAND_nointra       0.904694  0.097699  0.392435  0.393418
ltroutput_nointra  0.823925  0.114186  0.572315  0.572315
('mslr', True)
            

In [58]:
for s in [1,2,4,8,16,32]:
    print(s, '='*30)
    print_df(df2, s)

1 ==============================
(2019, False)
             Fairness                NDCG          
metric            DTR       EEL       DTR       EEL
learner                                            
PL_nointra   0.944041  0.050435  0.807647  0.785057
PPG_intra    0.931118  0.047584  0.813338  0.797277
PPG_nointra  0.933383  0.047410  0.810100  0.788975
(2020, False)
             Fairness                NDCG          
metric            DTR       EEL       DTR       EEL
learner                                            
PL_nointra   1.968721  0.173543  0.370768  0.363026
PPG_intra    1.982259  0.158058  0.364994  0.378790
PPG_nointra  1.980241  0.158162  0.368536  0.390616
2 ==============================
(2019, False)
             Fairness                NDCG          
metric            DTR       EEL       DTR       EEL
learner                                            
PL_nointra   0.947393  0.050707  0.810826  0.788554
PPG_intra    0.940532  0.047447  0.809882  0.791908
PPG_noin

In [9]:
def refine_df(df, metric, year):
    return df.loc[(df.metric==metric) & (df.year == year), sorted(list(set(df.columns)-set(('metric', 'year'))))].sort_values(by=['sessions', 'learner'])

df_ = df.loc[(df.sessions==32) & (df.true_labels==False)]
print('EEL 2019')
print(refine_df(df_, 'EEL', 2019).head(100))
print('EEL 2020')
print(refine_df(df_, 'EEL', 2020).head(100))
print('DTR 2019')
print(refine_df(df_, 'DTR', 2019).head(100))
print('DTR 2020')
print(refine_df(df_, 'DTR', 2020).head(100))

EEL 2019
      Fairness      NDCG  ind      learner    lr  samples  sessions  \
2559  0.029041  0.757846    0   PL_nointra  0.01       32        32   
2563  0.028032  0.758348    1   PL_nointra  0.01       32        32   
2567  0.027258  0.757838    2   PL_nointra  0.01       16        32   
2571  0.024298  0.758005    3   PL_nointra  0.01        8        32   
2575  0.028760  0.757778    4   PL_nointra  0.01       32        32   
2579  0.028516  0.759516    5   PL_nointra  0.01       32        32   
2583  0.025579  0.756579    6   PL_nointra  0.01        8        32   
2587  0.026532  0.757841    7   PL_nointra  0.01       16        32   
2591  0.026982  0.757819    8   PL_nointra  0.01       16        32   
2595  0.026735  0.758035    9   PL_nointra  0.01       16        32   
2599  0.028714  0.757752   10   PL_nointra  0.01       32        32   
2603  0.027769  0.758549   11   PL_nointra  0.01       32        32   
2607  0.028628  0.758734   12   PL_nointra  0.01       32        32 

In [42]:
def get_mean(df, metric, year, sessions):
    df_ = df.loc[(df.metric==metric) & (df.year == year) & (df.sessions==sessions), :]
    return df_.groupby(['learner', 'sessions', 'true_labels'])['Fairness', 'NDCG'].apply(np.mean)
    
for sessions in [1,2,4,8,16,32]:
    print('DTR 2019\n', get_mean(df2, 'DTR', 2019, sessions))
    print('DTR 2020\n', get_mean(df2, 'DTR', 2020, sessions))
    print('EEL 2019\n', get_mean(df2, 'EEL', 2019, sessions))
    print('EEL 2020\n', get_mean(df2, 'EEL', 2020, sessions))
    print('='*30)

/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a t

DTR 2019
                                   Fairness      NDCG
learner     sessions true_labels                    
PL_nointra  1        False        0.911150  0.793484
                     True         0.689018  0.892055
PPG_intra   1        False        0.835331  0.806363
                     True         0.330924  0.982059
PPG_nointra 1        False        0.836834  0.794397
                     True         0.334919  0.873962
DTR 2020
                                   Fairness      NDCG
learner     sessions true_labels                    
PL_nointra  1        False        2.057976  0.342638
                     True         1.795913  0.539321
PPG_intra   1        False        1.969781  0.379178
                     True         0.973346  0.931628
PPG_nointra 1        False        1.971472  0.341131
                     True         1.011618  0.412758
EEL 2019
                                   Fairness      NDCG
learner     sessions true_labels                    
PL_nointra  1   

EEL 2020
                                   Fairness      NDCG
learner     sessions true_labels                    
PL_nointra  32       False        0.098302  0.357185
                     True         0.014967  0.578785
PPG_intra   32       False        0.150882  0.388978
                     True         0.010413  0.961728
PPG_nointra 32       False        0.096016  0.364794
                     True         0.022942  0.424524


/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a t

In [45]:
def random_sample_PL()

DTR 2019
 learner      sessions  true_labels
PL_nointra   1         True           32
PPG_intra    1         True           32
PPG_nointra  1         True           32
dtype: int64
DTR 2019
 learner      sessions  true_labels
PL_nointra   2         True           32
PPG_intra    2         True           32
PPG_nointra  2         True           32
dtype: int64
DTR 2019
 learner      sessions  true_labels
PL_nointra   4         True           32
PPG_intra    4         True           32
PPG_nointra  4         True           32
dtype: int64
DTR 2019
 learner      sessions  true_labels
PL_nointra   8         True           32
PPG_intra    8         True           29
PPG_nointra  8         True           18
dtype: int64
DTR 2019
 learner      sessions  true_labels
PL_nointra   16        True           32
PPG_intra    16        True            8
PPG_nointra  16        True            7
dtype: int64
DTR 2019
 learner      sessions  true_labels
PL_nointra   32        True           32
PPG_intra

/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/avardas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a t